# Neural network Tuning

In [3]:
# Importing necesary packages
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


# part 1 - Tuning batch and epoch

In [7]:
# load the dataset for India diabetes
data = pd.read_csv('diabetes.csv')

In [8]:
#viewing data
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [9]:
# split into predictors and response
X = data.iloc[:,0:8] #predictors
Y = data.iloc[:,8] #response

In [10]:
# Function to create model,for KerasClassifier
def create_my_model():
    #defining my model
    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim=8, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))

    # Compile the model
    mymodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return mymodel

In [17]:
# create model
model = KerasClassifier(build_fn=create_my_model)

NameError: ignored

In [11]:
# define the grid search parameters
batchSize = [10, 20, 40, 60, 80, 100]
epochs = [10, 30, 50, 100, 200,400]

In [9]:
parameter_grid = dict(batch_size=batchSize, epochs=epochs)


mygrid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = mygrid.fit(X, Y)

Epoch 1/50
77/77 [==============================] - 2s 3ms/step - loss: 4.3230 - accuracy: 0.6016
Epoch 2/50
77/77 [==============================] - 0s 3ms/step - loss: 2.1673 - accuracy: 0.6497
Epoch 3/50
77/77 [==============================] - 0s 3ms/step - loss: 1.6892 - accuracy: 0.6576
Epoch 4/50
77/77 [==============================] - 0s 4ms/step - loss: 1.4881 - accuracy: 0.6680
Epoch 5/50
77/77 [==============================] - 0s 3ms/step - loss: 1.3236 - accuracy: 0.6641
Epoch 6/50
77/77 [==============================] - 0s 3ms/step - loss: 1.2151 - accuracy: 0.6536
Epoch 7/50
77/77 [==============================] - 0s 3ms/step - loss: 1.1130 - accuracy: 0.6680
Epoch 8/50
77/77 [==============================] - 0s 3ms/step - loss: 1.0528 - accuracy: 0.6549
Epoch 9/50
77/77 [==============================] - 0s 3ms/step - loss: 0.9768 - accuracy: 0.6667
Epoch 10/50
77/77 [==============================] - 0s 3ms/step - loss: 0.9413 - accuracy: 0.6693
Epoch 11/50
77/77 [

In [11]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.716146 using {'batch_size': 10, 'epochs': 50}


# part 2- Optimizing Activation Function

In [18]:
# Function to create model
def create_my_model(optimizer='adam'):
    # create model
    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim=8, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))
    # Compile model
    mymodel.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return mymodel

In [19]:
# create model
model = KerasClassifier(build_fn=create_my_model, epochs=50, batch_size=10)

In [20]:
# define the grid search parameters
optimizer = ['SGD','Adadelta', 'RMSprop', 'Adagrad','Adam']
parameter_grid = dict(optimizer=optimizer)

In [15]:
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

Epoch 1/50
77/77 [==============================] - 2s 5ms/step - loss: 11.4870 - accuracy: 0.4974
Epoch 2/50
77/77 [==============================] - 0s 4ms/step - loss: 7.9322 - accuracy: 0.4349
Epoch 3/50
77/77 [==============================] - 0s 3ms/step - loss: 6.1282 - accuracy: 0.4349
Epoch 4/50
77/77 [==============================] - 0s 3ms/step - loss: 4.4569 - accuracy: 0.4583
Epoch 5/50
77/77 [==============================] - 0s 3ms/step - loss: 3.2957 - accuracy: 0.4857
Epoch 6/50
77/77 [==============================] - 0s 3ms/step - loss: 2.4570 - accuracy: 0.4883
Epoch 7/50
77/77 [==============================] - 0s 3ms/step - loss: 1.8574 - accuracy: 0.5326
Epoch 8/50
77/77 [==============================] - 0s 3ms/step - loss: 1.4760 - accuracy: 0.5521
Epoch 9/50
77/77 [==============================] - 0s 3ms/step - loss: 1.2316 - accuracy: 0.5716
Epoch 10/50
77/77 [==============================] - 0s 3ms/step - loss: 1.0768 - accuracy: 0.5990
Epoch 11/50
77/77 

In [16]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.671875 using {'optimizer': 'Adam'}


# How can you improve accuracy of the model?

In [12]:
def create_model():
	# create model
  model = Sequential()
  model.add(Dense(12, input_shape=(8,), activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy',
                optimizer=optimizer, metrics=['accuracy'])

	# return model without compile
  return model
# fix random seed for reproducibility


In [13]:
import tensorflow as tf


In [21]:
seed = 7
tf.random.set_seed(seed)


In [22]:
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(model__optimizer=optimizer)


In [23]:
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

Epoch 1/50
77/77 [==============================] - 2s 3ms/step - loss: 7.2790 - accuracy: 0.5651
Epoch 2/50
77/77 [==============================] - 0s 3ms/step - loss: 3.5345 - accuracy: 0.6458
Epoch 3/50
77/77 [==============================] - 0s 3ms/step - loss: 3.1099 - accuracy: 0.6458
Epoch 4/50
77/77 [==============================] - 0s 3ms/step - loss: 2.7432 - accuracy: 0.6471
Epoch 5/50
77/77 [==============================] - 0s 3ms/step - loss: 2.4000 - accuracy: 0.6380
Epoch 6/50
77/77 [==============================] - 0s 3ms/step - loss: 2.1405 - accuracy: 0.6328
Epoch 7/50
77/77 [==============================] - 0s 3ms/step - loss: 1.7225 - accuracy: 0.6458
Epoch 8/50
77/77 [==============================] - 0s 3ms/step - loss: 1.4520 - accuracy: 0.6393
Epoch 9/50
77/77 [==============================] - 0s 3ms/step - loss: 1.0732 - accuracy: 0.6471
Epoch 10/50
77/77 [==============================] - 0s 3ms/step - loss: 0.8372 - accuracy: 0.6745
Epoch 11/50
77/77 [

We have reached 0.735 accuracy
